In [2]:
import folium
import pandas as pd
import requests
import json

In [3]:
#csv파일에서 랜덤으로 정보 들고오기
filename='jeju.csv'
df=pd.read_csv(filename,encoding='cp949')
df.head(2) #tail(밑에서2개), sample(섞어서2개)

,지역,읍면동,전화번호,주소,일반현황,청사현황,자치센터현황,데이터기준일자,Unnamed: 8
0,제주시,일도1동,064-728-4412,제주특별자치도 제주시 중앙로7길 15 (일도일동),인구 3146명 / 면적 0.33㎢ / 11통 61개반,지상 3층 / 연면적 585㎡,개소일 2000-12-20,2019-06-20,NaN
1,제주시,일도2동,064-728-4442,제주특별자치도 제주시 고마로 28 (일도이동),인구 38400명 / 면적2.18㎢ / 48통326반,지상 2층·지하 1층 / 연면적 1307.01,총면적 617.22㎡ / 개소일 2000-12-24,2019-06-20,NaN


In [4]:
#주소값을 가지고 위경도 가져오기

In [5]:
#kakaoAPI에 주소를 넣어서 위.경도 정보를 리턴하는 함수를 작성

def get_geo_code(address):
    REST_API_KEY='12c60ab21ec33500c86de172cbd60788'
    url=f'https://dapi.kakao.com/v2/local/search/keyword.json?query={address}&category_group_code=PO3'
    headers={'Authorization': f'KakaoAK {REST_API_KEY}'}
    res=requests.get(url,headers=headers)
    res=res.json()
    try:
        return res['documents'][0]['y'],res['documents'][0]['x'] #()하나,안하나 묶여서 나감
    except:
        return "결과없음"
    #return x,y #숫자형태로 리턴

In [6]:
get_geo_code('제주특별자치도 제주시 중앙로7길 15')

('33.5150737361162', '126.526342790758')

In [7]:
df['주소'].head()

0     제주특별자치도 제주시 중앙로7길 15 (일도일동)
1       제주특별자치도 제주시 고마로 28 (일도이동)
2    제주특별자치도 제주시 중앙로25길 17 (이도1동)
3       제주특별자치도 제주시 오복3길 8 (이도이동)
4       제주특별자치도 제주시 전농로 40 (삼도일동)
Name: 주소, dtype: object

In [8]:
df['주소'] = df['주소'].str.split('(').str.get(0).str.strip() #소괄호 기준으로 나눠줌, 앞쪽 0번 가져옴, 공백제거

In [9]:
df.sample(2)

,지역,읍면동,전화번호,주소,일반현황,청사현황,자치센터현황,데이터기준일자,Unnamed: 8
11,제주시,봉개동,064-728-4742,제주특별자치도 제주시 봉개북1길 15,인구 3046명 / 면적47.2㎢ / 7통30반,지하 1층·지상2층 / 연면적 770㎡,총면적 390㎡ / 개소일 2000-10-05,2019-06-20,NaN
30,서귀포시,표선면,064-760-4413,제주특별자치도 서귀포시 표선면 표선중앙로 74,인구 11167명 / 면적 135.172㎢ / 10개리 94개반,지상 2 층 지하 1층,총면적 214㎡ / 개소일 2006-08-16,2019-06-20,NaN


In [10]:
df.drop(columns='Unnamed: 8',inplace=True) #빈 데이터 지우기 #이미 지우고 위에 실행하면 에러 뜸ㅋ

In [11]:
df.sample()

,지역,읍면동,전화번호,주소,일반현황,청사현황,자치센터현황,데이터기준일자
19,제주시,한림읍,064-728-7628,제주시 한림읍 한림상로 132,인구 19925명 / 면적91.99㎢ / 21개리 56개자연마을 350개반,지상 2층 / 연면적 1066.33㎡,총면적 1066.33㎡ / 개소일 2007-07-19,2019-06-20


In [12]:
map = folium.Map([33.36423604300776, 126.53283910898249],zoom_start=10)
for index, row in df.iterrows():
    #print(index)
    #print(row['주소']) #주소 열을 키로 뽑기
    geodata = get_geo_code(row['주소'])
    #print(geodata)
    #print('=========')
    if geodata != None and geodata != "결과없음": #결과없음은 빼기
        folium.Marker(geodata,popup=row['읍면동']).add_to(map)
map

In [13]:
map = folium.Map([33.36423604300776, 126.53283910898249],zoom_start=10)
rfile = open('jeju-1.json','r').read()
jdata = json.loads(rfile)
folium.GeoJson(jdata,name='jdata').add_to(map)

In [14]:
rfile

'{\n"type": "FeatureCollection",\n                                                                                \n"features": [\n{ "type": "Feature","id":"Jeju", "properties": { "ID_0": 211, "ISO": "KOR", "NAME_0": "South Korea", "ID_1": 12, "NAME_1": "Jeju", "ID_2": 162, "NAME_2": "Jeju", "NL_NAME_2": "??? | ???", "VARNAME_2": null, "TYPE_2": "Si", "ENGTYPE_2": "City" }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ 126.160140991211165, 33.312915802002067 ], [ 126.160140991211165, 33.312637329101676 ], [ 126.16069793701206, 33.312637329101676 ], [ 126.16069793701206, 33.312362670898494 ], [ 126.160972595214957, 33.312362670898494 ], [ 126.160972595214957, 33.311805725097713 ], [ 126.16152954101608, 33.311805725097713 ], [ 126.16152954101608, 33.312084197998104 ], [ 126.162918090820767, 33.312084197998104 ], [ 126.162918090820767, 33.310695648193359 ], [ 126.162361145019872, 33.310695648193359 ], [ 126.162361145019872, 33.310417175292969 ], [ 126.16180419921875, 33.310

In [15]:
type(jdata)

dict

In [16]:
map

In [17]:
df

,지역,읍면동,전화번호,주소,일반현황,청사현황,자치센터현황,데이터기준일자
0,제주시,일도1동,064-728-4412,제주특별자치도 제주시 중앙로7길 15,인구 3146명 / 면적 0.33㎢ / 11통 61개반,지상 3층 / 연면적 585㎡,개소일 2000-12-20,2019-06-20
1,제주시,일도2동,064-728-4442,제주특별자치도 제주시 고마로 28,인구 38400명 / 면적2.18㎢ / 48통326반,지상 2층·지하 1층 / 연면적 1307.01,총면적 617.22㎡ / 개소일 2000-12-24,2019-06-20
2,제주시,이도1동,064-728-1530,제주특별자치도 제주시 중앙로25길 17,인구 7418명 / 면적 0.79㎢ / 16통 97개반,지상 2층 / 2층면적 205㎡,개소일 2000-12-28,2019-06-20
3,제주시,이도2동,064-755-2021,제주특별자치도 제주시 오복3길 8,인구 41428명 / 면적 5.4㎢ / 49통 323반,지하 1층·지상 3층 / 연면적 1223.52㎡,총면적 388.9㎡ / 개소일 2000-10-05,2019-06-20
4,제주시,삼도1동,064-728-4532,제주특별자치도 제주시 전농로 40,인구 14281명 / 면적0.87㎢ / 18통122반,지하 1층·지상3층 / 연면적 2070.37㎡,총면적 456.55㎡,2019-06-20
5,제주시,삼도2동,064-728-4562,제주특별자치도 제주시 관덕로6길 15,인구 9431명 / 면적0.83㎢ / 19통109반,지하 1층·지상3층 / 연면적 814㎡,총면적 368.69㎡ / 개소일 2000-12-30,2019-06-20
6,제주시,용담1동,064-728-4592,제주특별자치도 제주시 용남3길 11,인구 8368명 / 면적0.61㎢ / 15통102반,지상 3층 / 대지면적 8825㎡ / 건축면적 428.55,연면적 112049㎡ / 개소일 2000-12-24,2019-06-20
7,제주시,용담2동,064-728-1535,제주특별자치도 제주시 흥운길 27,인구 16701명 / 면적 4.94㎢ / 23통 155반,지상 3층 지하 1층 / 연면적 1046.13㎡,총면적 449.55㎡ / 개관일 2000-12-30,2019-06-20
8,제주시,건입동,064-728-1536,제주특별자치도 제주시 만덕로 18,인구 10850명 / 면적2.5㎢ / 20통113반,지상 4층 / 연면적 1192.37㎡,총면적 323.92㎡ / 개소일 2000-12-27,2019-06-20
9,제주시,화북동,064-728-4682,제주특별자치도 제주시 진남로6길 17,인구 25559('17.5월 현재) / 면적8.28㎢ / 33통 205반,지하 1층·지상3층 / 연면적 2375.80㎡,연면적 54㎡ / 개소일 2000-12-27,2019-06-20


In [18]:
#패턴 찾아서 인구수만 뽑기

In [19]:
#df['일반현황'].str.split('구').str.get(1).str.split('명').str.get(0).str.split('(').str.get(0).str.split('/').str.get(0)

In [20]:
df['인구']=df['일반현황'].str.split('/').str.get(0).str.split('(').str.get(0).str.strip().str.split(' ').str.get(-1).str.replace('명','').astype('int')
#명이라는 글자 기준으로 가져오겠다, 첫번째 값 가져옴, 콤마를 기준으로

In [21]:
df

,지역,읍면동,전화번호,주소,일반현황,청사현황,자치센터현황,데이터기준일자,인구
0,제주시,일도1동,064-728-4412,제주특별자치도 제주시 중앙로7길 15,인구 3146명 / 면적 0.33㎢ / 11통 61개반,지상 3층 / 연면적 585㎡,개소일 2000-12-20,2019-06-20,3146
1,제주시,일도2동,064-728-4442,제주특별자치도 제주시 고마로 28,인구 38400명 / 면적2.18㎢ / 48통326반,지상 2층·지하 1층 / 연면적 1307.01,총면적 617.22㎡ / 개소일 2000-12-24,2019-06-20,38400
2,제주시,이도1동,064-728-1530,제주특별자치도 제주시 중앙로25길 17,인구 7418명 / 면적 0.79㎢ / 16통 97개반,지상 2층 / 2층면적 205㎡,개소일 2000-12-28,2019-06-20,7418
3,제주시,이도2동,064-755-2021,제주특별자치도 제주시 오복3길 8,인구 41428명 / 면적 5.4㎢ / 49통 323반,지하 1층·지상 3층 / 연면적 1223.52㎡,총면적 388.9㎡ / 개소일 2000-10-05,2019-06-20,41428
4,제주시,삼도1동,064-728-4532,제주특별자치도 제주시 전농로 40,인구 14281명 / 면적0.87㎢ / 18통122반,지하 1층·지상3층 / 연면적 2070.37㎡,총면적 456.55㎡,2019-06-20,14281
5,제주시,삼도2동,064-728-4562,제주특별자치도 제주시 관덕로6길 15,인구 9431명 / 면적0.83㎢ / 19통109반,지하 1층·지상3층 / 연면적 814㎡,총면적 368.69㎡ / 개소일 2000-12-30,2019-06-20,9431
6,제주시,용담1동,064-728-4592,제주특별자치도 제주시 용남3길 11,인구 8368명 / 면적0.61㎢ / 15통102반,지상 3층 / 대지면적 8825㎡ / 건축면적 428.55,연면적 112049㎡ / 개소일 2000-12-24,2019-06-20,8368
7,제주시,용담2동,064-728-1535,제주특별자치도 제주시 흥운길 27,인구 16701명 / 면적 4.94㎢ / 23통 155반,지상 3층 지하 1층 / 연면적 1046.13㎡,총면적 449.55㎡ / 개관일 2000-12-30,2019-06-20,16701
8,제주시,건입동,064-728-1536,제주특별자치도 제주시 만덕로 18,인구 10850명 / 면적2.5㎢ / 20통113반,지상 4층 / 연면적 1192.37㎡,총면적 323.92㎡ / 개소일 2000-12-27,2019-06-20,10850
9,제주시,화북동,064-728-4682,제주특별자치도 제주시 진남로6길 17,인구 25559('17.5월 현재) / 면적8.28㎢ / 33통 205반,지하 1층·지상3층 / 연면적 2375.80㎡,연면적 54㎡ / 개소일 2000-12-27,2019-06-20,25559


In [22]:
#["jeju","Seogwipo"]
df['지역'] = df['지역'].replace(['제주시','서귀포시'],["Jeju","Seogwipo"]) #str없이 replace는 완전히 일치하는 것만 뽑음. 부분일치 못찾음.

In [23]:
df1 = df.groupby(['지역']).sum()
df1

,인구
지역,
Jeju,462396
Seogwipo,167641


In [24]:
map = folium.Map([33.36423604300776, 126.53283910898249],zoom_start=10)
rfile = open('jeju-1.json','r').read()
jdata = json.loads(rfile)
folium.Choropleth(jdata,
                 data=df1['인구'],
                 columns=[df.index,df1['인구']],
                 key_on='feature.id',
                 fill_color='PuRd',
                 legend_name='제주 인구').add_to(map)
map

In [25]:
df2 = df.groupby(['읍면동']).sum()
df2.sample(2)

,인구
읍면동,
이호동,4104
용담2동,16701


In [26]:
map = folium.Map([33.36423604300776, 126.53283910898249],zoom_start=10)
rfile = open('jeju.json','r').read()
jdata = json.loads(rfile)
folium.Choropleth(jdata,
                 data=df2['인구'],
                 columns=[df2.index,df2['인구']],
                 key_on='feature.id',
                 fill_color='YlGn',
                 fill_opacity=0.7,
                 line_opacity=0.3,
                 legend_name='제주 인구').add_to(map)
map

In [27]:
#래핑 된 결과 값에서 잘 못 나온 정보를 제외하고 제주도만 나오게 하기
#"sidonm": "제주특별자치도" 

In [35]:
rfile = open('HangJeongDong_ver20220309.geojson','r',encoding='utf-8').read()
jdata = json.loads(rfile)
jsondata_jeju = {'type':'FeatureCollection'}
jsondata_pick = []
for item in jdata['features']:
    #print(item)
    if item['properties']['sidonm'] == '제주특별자치도':
        print(item['properties']['emd_nm'])
        item['id']=item['properties']['emd_nm']
        jsondata_pick.append(item)
jsondata_jeju['features']=jsondata_pick

한림읍
애월읍
구좌읍
조천읍
한경면
추자면
우도면
일도1동
일도2동
이도1동
이도2동
삼도1동
삼도2동
용담1동
용담2동
건입동
화북동
삼양동
봉개동
아라동
오라동
연동
노형동
외도동
이호동
도두동
대정읍
남원읍
성산읍
안덕면
표선면
송산동
정방동
중앙동
천지동
효돈동
영천동
동홍동
서홍동
대륜동
대천동
중문동
예래동


In [38]:
#jsondata_jeju

In [40]:
map = folium.Map([33.36423604300776, 126.53283910898249],zoom_start=10)
#rfile = open('jeju.json','r').read()
#jdata = json.loads(rfile)
c=folium.Choropleth(jsondata_jeju,
                 data=df2['인구'],
                 columns=[df2.index,df2['인구']],
                 key_on='feature.id',
                 fill_color='YlGn',
                 fill_opacity=0.7,
                 line_opacity=0.3,
                 legend_name='제주 인구').add_to(map)
c.geojson.add_child(
    folium.features.GeoJsonTooltip(['emd_nm'],labels=False)
)
map